<a href="https://colab.research.google.com/github/KevinWang676/Bark-Voice-Cloning/blob/main/%E6%9C%80%E6%96%B0%E4%B8%AD%E6%96%87%E5%A3%B0%E9%9F%B3%E5%85%8B%E9%9A%86%E5%9C%A8%E7%BA%BF%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 全新中文声音克隆 Voice Cloning for Chinese Speech

## 配置环境 Set up

In [6]:
! nvidia-smi # 需要使用GPU运行

Thu Aug 31 03:55:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
! pip install openai-whisper
! pip install modelscope
! pip install tts-autolabel -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
! pip install typeguard==2.3.1
! pip install sox
! pip install bitstring
! pip install pysptk --no-build-isolation
! pip install kantts -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
! pip install pytorch_wavelets
! pip install tensorboardX
! git clone https://github.com/fbcotter/pytorch_wavelets
! pip install matplotlib
! pip install numpy==1.22.0

Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
  Using cached numpy-1.23.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 13.0.0 which is incompatible.


Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html
fatal: destination path 'pytorch_wavelets' already exists and is not an empty directory.
  Using cached numpy-1.22.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.1
    Uninstalling numpy-1.23.1:
      Successfully uninstalled numpy-1.23.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.1 requires numpy!=1.22.0,!=1.22.1,!=1.22.2,>=1.20.3, but you have numpy 1.22.0 which is incompatible.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 13.0.0 which is incompatible.
plotnine 0.12.2 requires numpy>=1.23.0, but you have numpy 1.22.0 which is incompatible.


In [8]:
!apt-get install sox

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sox is already the newest version (14.4.2+git20190427-2+deb11u2build0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [4]:
cd pytorch_wavelets

/content/pytorch_wavelets


In [5]:
pip install .

Processing /content/pytorch_wavelets
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-wavelets: filename=pytorch_wavelets-1.3.0-py3-none-any.whl size=54852 sha256=1766817273f5e997676835b2f954ecc0a361190525d6f5e4936f12ba4062e296
  Stored in directory: /tmp/pip-ephem-wheel-cache-c8dgc3ad/wheels/f6/77/89/ac0462961f441a5dd4d226216613ef64f3eb2c7f2883efcebf
Successfully built pytorch-wavelets
  Attempting uninstall: pytorch-wavelets
    Found existing installation: pytorch-wavelets 1.3.0
    Uninstalling pytorch-wavelets-1.3.0:
      Successfully uninstalled pytorch-wavelets-1.3.0


In [ ]:
import os
os._exit(00) # 重启notebook

In [6]:
import sox

## 对音频切片处理

In [7]:
import subprocess
from pathlib import Path
import librosa
from scipy.io import wavfile
import numpy as np
import torch
import csv
import whisper

def split_long_audio(model, filepaths, save_dir="data_dir", out_sr=44100):
    if isinstance(filepaths, str):
        filepaths = [filepaths]

    for file_idx, filepath in enumerate(filepaths):

        save_path = Path(save_dir)
        save_path.mkdir(exist_ok=True, parents=True)

        print(f"Transcribing file {file_idx}: '{filepath}' to segments...")
        result = model.transcribe(filepath, word_timestamps=True, task="transcribe", beam_size=5, best_of=5)
        segments = result['segments']

        wav, sr = librosa.load(filepath, sr=None, offset=0, duration=None, mono=True)
        wav, _ = librosa.effects.trim(wav, top_db=20)
        peak = np.abs(wav).max()
        if peak > 1.0:
            wav = 0.98 * wav / peak
        wav2 = librosa.resample(wav, orig_sr=sr, target_sr=out_sr)
        wav2 /= max(wav2.max(), -wav2.min())

        for i, seg in enumerate(segments):
            start_time = seg['start']
            end_time = seg['end']
            wav_seg = wav2[int(start_time * out_sr):int(end_time * out_sr)]
            wav_seg_name = f"{file_idx}_{i}.wav"
            out_fpath = save_path / wav_seg_name
            wavfile.write(out_fpath, rate=out_sr, data=(wav_seg * np.iinfo(np.int16).max).astype(np.int16))

In [8]:
whisper_size = "medium"
whisper_model = whisper.load_model(whisper_size)

100%|██████████████████████████████████████| 1.42G/1.42G [00:06<00:00, 219MiB/s]


## 需要新建两个文件夹，分别是："output_training_data", "pretrain_work_dir"

In [9]:
from modelscope.tools import run_auto_label

import os
from modelscope.models.audio.tts import SambertHifigan
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

from modelscope.metainfo import Trainers
from modelscope.trainers import build_trainer
from modelscope.utils.audio.audio_utils import TtsTrainType

pretrained_model_id = 'damo/speech_personal_sambert-hifigan_nsf_tts_zh-cn_pretrain_16k'

dataset_id = "./output_training_data/"
pretrain_work_dir = "./pretrain_work_dir/"



2023-08-31 04:00:15,831 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-08-31 04:00:15,835 - modelscope - INFO - TensorFlow version 2.12.0 Found.
2023-08-31 04:00:15,836 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2023-08-31 04:00:15,838 - modelscope - INFO - No valid ast index found from /root/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2023-08-31 04:00:15,890 - modelscope - INFO - Loading done! Current index file version is 1.8.4, with md5 d1f43cd279c8b7665e164b7d88d6f3a9 and a total number of 902 components indexed


In [27]:
def auto_label(audio):
  try:
    split_long_audio(whisper_model, audio, "test_wavs")
    os.makedirs("output_training_data", exist_ok=True)
    input_wav = "./test_wavs/"
    output_data = "./output_training_data/"
    ret, report = run_auto_label(input_wav=input_wav, work_dir=output_data, resource_revision="v1.0.7")

  except Exception:
    pass
  return "标注成功"

In [28]:
def train(a):
  try:
    os.makedirs("pretrain_work_dir", exist_ok=True)

    train_info = {
        TtsTrainType.TRAIN_TYPE_SAMBERT: {  # 配置训练AM（sambert）模型
            'train_steps': 52,               # 训练多少个step
            'save_interval_steps': 50,       # 每训练多少个step保存一次checkpoint
            'log_interval': 10               # 每训练多少个step打印一次训练日志
        }
    }

    # 配置训练参数，指定数据集，临时工作目录和train_info
    kwargs = dict(
        model=pretrained_model_id,                  # 指定要finetune的模型
        model_revision = "v1.0.6",
        work_dir=pretrain_work_dir,                 # 指定临时工作目录
        train_dataset=dataset_id,                   # 指定数据集id
        train_type=train_info                       # 指定要训练类型及参数
    )

    trainer = build_trainer(Trainers.speech_kantts_trainer,
                            default_args=kwargs)

    trainer.train()

  except Exception:
    pass

  return "训练完成"


In [29]:
import random

def infer(text):

  model_dir = os.path.abspath("./pretrain_work_dir")

  custom_infer_abs = {
      'voice_name':
      'F7',
      'am_ckpt':
      os.path.join(model_dir, 'tmp_am', 'ckpt'),
      'am_config':
      os.path.join(model_dir, 'tmp_am', 'config.yaml'),
      'voc_ckpt':
      os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan', 'ckpt'),
      'voc_config':
      os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan',
              'config.yaml'),
      'audio_config':
      os.path.join(model_dir, 'data', 'audio_config.yaml'),
      'se_file':
      os.path.join(model_dir, 'data', 'se', 'se.npy')
  }
  kwargs = {'custom_ckpt': custom_infer_abs}

  model_id = SambertHifigan(os.path.join(model_dir, "orig_model"), **kwargs)

  inference = pipeline(task=Tasks.text_to_speech, model=model_id)
  output = inference(input=text)

  filename = str(random.randint(1, 1000000000000))

  with open(filename + "myfile.wav", mode='bx') as f:
      f.write(output["output_wav"])
  return filename + "myfile.wav"


In [23]:
pip install gradio

In [24]:
import gradio as gr

In [30]:
with app:
    gr.Markdown("# <center>🥳🎶🎡 - Sambert中文声音克隆</center>")
    gr.Markdown("## <center>🌟 - 训练3分钟，推理5秒钟，中英自然发音 </center>")
    gr.Markdown("### <center>🌊 - 更多精彩应用，敬请关注[滔滔AI](http://www.talktalkai.com)；滔滔AI，为爱滔滔！💕</center>")

    with gr.Row():
      inp1 = gr.Audio(type="filepath", label="请上传一段音频")
      out1 = gr.Textbox(label="标注情况", lines=1, interactive=False)

      out2 = gr.Textbox(label="训练情况", lines=1, interactive=False)
      inp2 = gr.Textbox(label="文本", lines=3)
      out3 = gr.Audio(type="filepath", label="合成的音频")
      btn1 = gr.Button("标注数据")
      btn2 = gr.Button("训练")
      btn3 = gr.Button("推理")

      btn1.click(auto_label, inp1, out1)
      btn2.click(train, out1, out2)
      btn3.click(infer, inp2, out3)

    gr.Markdown("### <center>注意❗：请不要生成会对个人以及组织造成侵害的内容，此程序仅供科研、学习及个人娱乐使用。</center>")
    gr.HTML('''
        <div class="footer">
                    <p>🌊🏞️🎶 - 江水东流急，滔滔无尽声。 明·顾璘
                    </p>
        </div>
    ''')
app.launch(show_error=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [32]:
infer("欢迎使用滔滔智能的声音克隆产品")

2023-08-31 04:42:04,355 - modelscope - INFO - am_config=/content/pytorch_wavelets/pretrain_work_dir/tmp_am/config.yaml voc_config=/content/pytorch_wavelets/pretrain_work_dir/orig_model/basemodel_16k/hifigan/config.yaml
2023-08-31 04:42:04,356 - modelscope - INFO - audio_config=/content/pytorch_wavelets/pretrain_work_dir/data/audio_config.yaml
2023-08-31 04:42:04,357 - modelscope - INFO - am_ckpts=OrderedDict([(2400000, '/content/pytorch_wavelets/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400000.pth'), (2400050, '/content/pytorch_wavelets/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400050.pth')])
2023-08-31 04:42:04,361 - modelscope - INFO - voc_ckpts=OrderedDict([(2400000, '/content/pytorch_wavelets/pretrain_work_dir/orig_model/basemodel_16k/hifigan/ckpt/checkpoint_2400000.pth')])
2023-08-31 04:42:04,362 - modelscope - INFO - se_path=/content/pytorch_wavelets/pretrain_work_dir/data/se/se.npy se_model_path=/content/pytorch_wavelets/pretrain_work_dir/orig_model/se.onnx
2023-08-31 04:42:04,3

Removing weight norm...


'734589414357myfile.wav'